# Projet électif python: Basket Data

Ce notebook sert de rendu à notre projet d'analyse de données sur la NBA.



---


On commence par importer les librairies utilisés pour l'ensemble du projet:


*   numpy pour la gestion des grands nombres ainsi que certaines fonctions mathématiques.
*   panda pour la récupération des données depuis un fichier `.csv`.
*   plotly pour la représentation graphique des statistiques souhaitées avec un contrôle accru de l'infographie.

In [37]:
%pip install pandas numpy plotly requests beautifulsoup4

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import requests
from bs4 import BeautifulSoup

Note: you may need to restart the kernel to use updated packages.


Le code ci-dessous permet de récupérer le fichier .csv source contenant les stats de l'ensemble des joueurs de la NBA (ici la saison 2022-23) et récupère les informations relatives à un joueur précis.

In [38]:
# Chemin vers le fichier CSV
chemin_fichier_csv = "NBA Stats 202324 All Stats  NBA Player Props Tool.csv"
# Importer les données depuis le fichier CSV
donnees = pd.read_csv(chemin_fichier_csv)
input_joueur = input("Veuillez saisir le nom du joueur : ")
attributs = ['PPG', 'RPG', 'APG', 'BPG', 'SPG', 'TPG', 'TS%']

def trouver_joueur(input_nom, dataframe):
    """
    Cette fonction recherche le meilleur résultat correspondant à l'input_nom parmi les noms de joueur disponibles dans le dataframe.
    """
    meilleur_resultat = None
    meilleur_score = 0
    
    # Convertir l'input en minuscules pour la recherche sans distinguer la casse
    input_nom = input_nom.lower()
    
    # Parcourir les noms de joueur disponibles dans le dataframe
    for joueur in dataframe['NAME']:
        # Convertir le nom du joueur en minuscules pour la comparaison sans distinguer la casse
        joueur_lower = joueur.lower()
        
        # Calculer le score de correspondance entre l'input et le nom du joueur
        score = 0
        for mot in input_nom.split():
            if mot in joueur_lower:
                score += 1
        
        # Mettre à jour le meilleur résultat
        if score > meilleur_score:
            meilleur_score = score
            meilleur_resultat = joueur
    
    return meilleur_resultat

def recup_stats_joueur(nom_joueur, attributs):
    stats = {}
    # On récupère les lignes correspondant au joueur recherché
    donnees_joueur = donnees[donnees['NAME'] == nom_joueur]
    if not donnees_joueur.empty:
        # Boucler sur chaque attribut
        for attribut in attributs:
            # Filtrer: on retire les valeurs non valides (NaN)
            valeurs_valides = donnees_joueur[attribut].dropna()

            # Vérifier qu'il reste des valeurs valides après le filtrage
            if not valeurs_valides.empty:
                # Calculer la moyenne des valeurs valides et stocker dans le dictionnaire
                moyenne_attribut = valeurs_valides.mean()
                stats[attribut] = moyenne_attribut
            else:
                print(f"Aucune valeur valide trouvée pour la stat: {attribut}.")
    else:
        print("Le joueur", nom_joueur, "n'a pas été trouvé dans les données.")
    return stats

# Appeler la fonction pour récupérer les statistiques du joueur
joueur_trouve = trouver_joueur(input_joueur, donnees)
stats_joueur = recup_stats_joueur(joueur_trouve, attributs)

# Afficher les statistiques du joueur
print(f"Stats de {joueur_trouve}:")
for attribut, stat in stats_joueur.items():
    print(f"{attribut}: {stat}")

Stats de LeBron James:
PPG: 25.7
RPG: 7.3
APG: 8.3
BPG: 0.5
SPG: 1.3
TPG: 3.4
TS%: 0.63


Cette fonction prend en entrée un `DataFrame` contenant les données des joueurs ainsi qu'une liste d'attributs.
Elle récupère les données pour chaque joueur dans le `DataFrame`, calcule la moyenne de chaque attribut et renvoie un dictionnaire contenant les moyennes de chaque attribut.

In [39]:
def calculer_moyenne_attributs_joueurs(dataframe, attributs):
    moyennes = {}

    # Boucler sur chaque attribut
    for attribut in attributs:
        # Filtrer: on retire les valeurs non valides (NaN)
        valeurs_valides = dataframe[attribut].dropna()

        # Vérifier qu'il reste des valeurs valides après le filtrage
        if len(valeurs_valides) > 0:
            moyenne_attribut = np.mean(valeurs_valides)

            # Ajouter la moyenne de l'attribut au dictionnaire des moyennes
            moyennes[attribut] = moyenne_attribut
        else:
            print(f"Aucune valeur valide trouvée pour la stat: {attribut}.")

    return moyennes

# Calculer la moyenne des attributs pour l'ensemble des joueurs de la liste
moyennes_globales = calculer_moyenne_attributs_joueurs(donnees, attributs)
print("Moyennes globales des stats de l'ensemble des joueurs:")
for attribut, moyenne in moyennes_globales.items():
    print(f"{attribut}: {moyenne}")

Moyennes globales des stats de l'ensemble des joueurs:
PPG: 8.148858447488584
RPG: 3.2777777777777777
APG: 1.9528158295281584
BPG: 0.38843226788432267
SPG: 0.5881278538812785
TPG: 0.9555555555555555
TS%: 0.5372115677321156


Cette fonction reprend le principe de la fonction précédente et calcule les moyennes des statistiques pour les joueurs du même poste que celui du joueur spécifié.

In [40]:
def moyenne_attributs_meme_poste(dataframe, nom_joueur, attributs):
    # Filtrer les données pour n'inclure que les joueurs du même poste que celui spécifié
    poste_joueur = dataframe.loc[dataframe['NAME'] == nom_joueur, 'POS'].iloc[0]
    donnees_meme_poste = dataframe[dataframe['POS'] == poste_joueur]
    # Calcul des moyennes des attributs pour les joueurs
    moyennes_meme_poste = calculer_moyenne_attributs_joueurs(donnees_meme_poste, attributs)
    return moyennes_meme_poste

# Calculer les moyennes des attributs pour les joueurs du même poste que celui du joueur spécifié
moyennes_meme_poste = moyenne_attributs_meme_poste(donnees, joueur_trouve, attributs)
# Afficher les moyennes calculées
print("Moyennes des stats pour les joueurs du même poste que", joueur_trouve)
for attribut, moyenne in moyennes_meme_poste.items():
    print(f"{attribut}: {moyenne}")

Moyennes des stats pour les joueurs du même poste que LeBron James
PPG: 8.354999999999999
RPG: 3.4479166666666665
APG: 1.5441666666666667
BPG: 0.40166666666666667
SPG: 0.5670833333333334
TPG: 0.91
TS%: 0.5370124999999999


Pondération par le nombre de minute jouées par match

In [41]:
def get_mpg_joueur(dataframe, nom_joueur):
    mpg_joueur = dataframe.loc[dataframe['NAME'] == nom_joueur, 'MPG'].iloc[0]
    return mpg_joueur
get_mpg_joueur(donnees, joueur_trouve)
def get_mpg_joueur_global(dataframe):
    moyenne_mpg = dataframe['MPG'].mean()
    return moyenne_mpg
get_mpg_joueur_global(donnees)
def get_mpg_joueur_poste(dataframe, nom_joueur):
    poste_joueur = dataframe.loc[dataframe['NAME'] == nom_joueur, 'POS'].iloc[0]
    mpg_poste = dataframe.loc[dataframe['POS'] == poste_joueur, 'MPG'].mean()
    return mpg_poste
get_mpg_joueur_poste(donnees, joueur_trouve)       

def ponderation_par_minute(donnees_normalisees_joueur, mpg_joueur):
    donnees_joueur_pond = {attribut: valeur / mpg_joueur for attribut, valeur in donnees_normalisees_joueur.items()}
    return donnees_joueur_pond

# Appel de la fonction en passant les données normalisées du joueur en tant qu'argument
ponderation_par_minute(stats_joueur, get_mpg_joueur(donnees, joueur_trouve))
ponderation_par_minute(moyennes_globales, get_mpg_joueur_global(donnees))
ponderation_par_minute(moyennes_meme_poste, get_mpg_joueur_poste(donnees, joueur_trouve))

{'PPG': 0.44784919819538127,
 'RPG': 0.1848170813418502,
 'APG': 0.08277125117255552,
 'BPG': 0.021530352436682006,
 'SPG': 0.030397105462991923,
 'TPG': 0.04877830883995177,
 'TS%': 0.028785232501005047}

La fonction ci-dessous permet de normaliser les statistiques récupérées pour le joueur étudié ainsi que les moyennes calculées. Pour ce faire, on recherche la valeur maximale de chaque stat, tous joueurs confondus, puis on calcul le ratio entre ces deux valeurs pour obtenir un pourcentage.
N.B.: ce traitement permet d'aboutir à des stats en % pour obtenir un diagramme homogène.

In [42]:
def normaliser_donnees(dataframe, joueur, attributs, stats):
    # Créer un dictionnaire pour stocker les valeurs maximales de chaque attribut
    max_valeurs = {attribut: dataframe[attribut].max() for attribut in attributs}
    # Normaliser les données brutes en pourcentage par rapport à la valeur maximale
    donnees_normalisees = {}
    for attribut in attributs:
        # Normaliser les données brutes du joueur
        valeur_joueur = stats[attribut]
        pourcentage_joueur = np.round((valeur_joueur / max_valeurs[attribut]) * 100, 1)  # Arrondir à une décimale
        # Stocker les valeurs normalisées dans le dictionnaire
        donnees_normalisees[attribut] = pourcentage_joueur
    return donnees_normalisees

# Utilisation de la fonction pour normaliser les données pour le joueur spécifié
donnees_normalisees_joueur = normaliser_donnees(donnees, joueur_trouve, attributs, stats_joueur)
donnees_normalisees_globales = normaliser_donnees(donnees, joueur_trouve, attributs, moyennes_globales)
donnees_normalisees_poste = normaliser_donnees(donnees, joueur_trouve, attributs, moyennes_meme_poste)

Normalisation avec pondération

In [43]:
donnees_normalisees_pondere_joueur=normaliser_donnees(donnees, joueur_trouve, attributs, ponderation_par_minute(stats_joueur, get_mpg_joueur(donnees, joueur_trouve)))
donnees_normalisees_pondere_globales=normaliser_donnees(donnees, joueur_trouve, attributs, ponderation_par_minute(moyennes_globales, get_mpg_joueur_global(donnees)))
donnees_normalisees_pondere_poste=normaliser_donnees(donnees, joueur_trouve, attributs, ponderation_par_minute(moyennes_meme_poste, get_mpg_joueur_poste(donnees, joueur_trouve)))




---


La fonction suivante permet la génération d'un diagramme de Kiviat représentant les statistiques normalisées du joueur séléctionné ainsi que celles des joueurs de toute la ligue, et celles des joueurs du même poste (moyennes). L'objectif est de proposer une comparaison pertinente pour un joueur.

**Remarque :** *Les statistiques affichées sont représentées en % et ne représentent donc pas les valeurs réelles.*

In [44]:
import plotly.graph_objects as go

def diagramme_kiviat(data_joueur, data_moyenne_globale, data_moyenne_poste, joueur):
    """
    Cette fonction crée un diagramme de Kiviat pour visualiser les données normalisées des attributs.
    """
    # Extraire les noms des attributs
    attributs = list(data_joueur.keys())

    # Extraire les valeurs normalisées pour le joueur, la moyenne globale et la moyenne du poste
    valeurs_joueur = list(data_joueur.values())
    valeurs_moyenne_globale = list(data_moyenne_globale.values())
    valeurs_moyenne_poste = list(data_moyenne_poste.values())

    # Créer un diagramme de Kiviat avec Plotly
    fig = go.Figure()

    # Ajouter les traces pour le joueur, la moyenne globale et la moyenne du poste
    fig.add_trace(go.Scatterpolar(
        r=valeurs_joueur,
        theta=attributs,
        fill='toself',
        name=f'{joueur}',
        line=dict(color='blue')
    ))
    fig.add_trace(go.Scatterpolar(
        r=valeurs_moyenne_globale,
        theta=attributs,
        fill='toself',
        name='Moyenne globale',
        line=dict(color='green')
    ))
    fig.add_trace(go.Scatterpolar(
        r=valeurs_moyenne_poste,
        theta=attributs,
        fill='toself',
        name='Moyenne du poste',
        line=dict(color='orange')
    ))

    # Ajouter un titre et ajuster la mise en page
    fig.update_layout(
        title=f"Comparaison des stats pour {joueur}",
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 100]  # Plage de l'axe radial en pourcentage
            )
        )
    )

    # Afficher le diagramme
    fig.show()

# Utiliser la fonction pour créer le diagramme de Kiviat
diagramme_kiviat(donnees_normalisees_joueur, donnees_normalisees_globales, donnees_normalisees_poste, joueur_trouve)

Diagramme de kiviat pondéré

In [45]:
diagramme_kiviat(donnees_normalisees_pondere_joueur, donnees_normalisees_pondere_globales, donnees_normalisees_pondere_poste, joueur_trouve)

In [46]:
# Trouver l'équipe du joueur
equipe_joueur = donnees[donnees['NAME'] == joueur_trouve]
equipe_joueur = equipe_joueur['TEAM']
# Afficher l'équipe du joueur
print("Équipe de", joueur_trouve + ":", equipe_joueur)


chemin_equipe_csv = "NBA Stats 202223 All Stats  NBA Player Props Tool.csv"
# Importer les données depuis le fichier team CSV
team = pd.read_csv(chemin_equipe_csv)
valeurs = ['PPG', 'oEFF', 'dEFF', 'W', 'L', 'pDIFF']


# Récupérer les joueurs par équipe
def meme_equipe(dataframe, nom_joueur):
    # Filtrer les données pour n'inclure que les joueurs de la même équipe que celui spécifié
    equipe_joueur = dataframe.loc[dataframe['NAME'] == nom_joueur, 'TEAM'].iloc[0]
    donnees_meme_equipe = dataframe[dataframe['TEAM'] == equipe_joueur]
    return donnees_meme_equipe
# Afficher les noms des joueurs de la même équipe que celui spécifié
print("Les joueurs de l'équipe de ", joueur_trouve, "sont les suivants :")
equipe_joueur = meme_equipe(donnees, joueur_trouve)
print(equipe_joueur['NAME'])


def cinq_majeur(dataframe, nom_joueur):
    # Récupérer les données des joueurs de la même équipe que celui spécifié
    equipe = meme_equipe(dataframe, nom_joueur)
    # Trier les joueurs par ordre décroissant de temps de jeu moyen (MPG)
    equipe_triee = equipe.sort_values(by='MPG', ascending=False)
    # Sélectionner les cinq premiers joueurs avec le temps de jeu moyen le plus élevé
    cinq_majeur = equipe_triee.head(5)
    return cinq_majeur
cinq_joueurs = cinq_majeur(donnees, joueur_trouve)
print("Les cinq joueurs ayant le temps de jeu moyen le plus élevé dans l'équipe sont :")
print(cinq_joueurs[['NAME', 'MPG']])

Équipe de LeBron James: 14    Lal
Name: TEAM, dtype: object
Les joueurs de l'équipe de  LeBron James sont les suivants :
14            LeBron James
18           Anthony Davis
68        D'Angelo Russell
87           Austin Reaves
127          Rui Hachimura
222         Taurean Prince
293         Christian Wood
306      Spencer Dinwiddie
356            Cam Reddish
371      Jarred Vanderbilt
422           Jaxson Hayes
431           Max Christie
501           Gabe Vincent
565            D'Moi Hodge
581    Jalen Hood-Schifino
588        Colin Castleton
591          Dylan Windler
602            Skylar Mays
618             Alex Fudge
638          Maxwell Lewis
639        Harry Giles III
Name: NAME, dtype: object
Les cinq joueurs ayant le temps de jeu moyen le plus élevé dans l'équipe sont :
                 NAME   MPG
18      Anthony Davis  35.5
14       LeBron James  35.3
68   D'Angelo Russell  32.7
87      Austin Reaves  32.1
222    Taurean Prince  27.0


Cette fonction récupère les joueurs d'une équipe spécifique à partir des données.

In [47]:
correspondance_equipes = {
    'Phi': 'Philadelphia 76ers',
    'Dal': 'Dallas Mavericks',
    'Por': 'Portland Trail Blazers',
    'Okc': 'Oklahoma City Thunder',
    'Mil': 'Milwaukee Bucks',
    'Bos': 'Boston Celtics',
    'Bro': 'Brooklyn Nets',
    'Gol': 'Golden State Warriors',
    'Lal': 'Los Angeles Lakers',
    'Cle': 'Cleveland Cavaliers',
    'Pho': 'Phoenix Suns',
    'Mem': 'Memphis Grizzlies',
    'Atl': 'Atlanta Hawks',
    'Nor': 'New Orleans Pelicans',
    'Uta': 'Utah Jazz',
    'Nyk': 'New York Knicks',
    'Sac': 'Sacramento Kings',
    'Chi': 'Chicago Bulls',
    'Min': 'Minnesota Timberwolves',
    'Den': 'Denver Nuggets',
    'Tor': 'Toronto Raptors',
    'Lac': 'Los Angeles Clippers',
    'Cha': 'Charlotte Hornets',
    'Was': 'Washington Wizards',
    'Mia': 'Miami Heat',
    'Hou': 'Houston Rockets',
    'San': 'San Antonio Spurs',
    'Det': 'Detroit Pistons',
    'Ind': 'Indiana Pacers',
    'Orl': 'Orlando Magic'
}

def trouver_equipe(input_nom, equipes_disponibles):
    """
    Cette fonction recherche le meilleur résultat correspondant à l'input_nom parmi les noms d'équipe disponibles.
    """
    meilleur_resultat = None
    meilleur_score = 0
    
    # Convertir l'input en minuscules pour la recherche sans distinguer la casse
    input_nom = input_nom.lower()
    
    # Parcourir les noms d'équipe disponibles
    for equipe_abrege in equipes_disponibles:
        equipe_complet = correspondance_equipes.get(equipe_abrege)
        if not equipe_complet:
            continue
        
        # Convertir le nom de l'équipe en minuscules pour la comparaison sans distinguer la casse
        equipe_lower = equipe_complet.lower()
        
        # Calculer le score de correspondance entre l'input et le nom de l'équipe
        score = 0
        for mot in input_nom.split():
            if mot in equipe_lower:
                score += 1
        
        # Mettre à jour le meilleur résultat
        if score > meilleur_score:
            meilleur_score = score
            meilleur_resultat = equipe_abrege
    
    return meilleur_resultat


def liste_equipes(dataframe):
    # Récupérer les noms uniques des équipes dans la colonne "Equipe"
    equipes = dataframe['TEAM'].unique().tolist()
    return equipes

def joueurs_par_equipe(dataframe, nom_equipe):
    # Filtrer les données pour n'inclure que les joueurs de l'équipe spécifiée
    joueurs_equipe = dataframe[dataframe['TEAM'] == nom_equipe]
    return joueurs_equipe


equipes_disponibles = liste_equipes(team)
input_nom = input("Entrez le nom de l'équipe : ")
equipe_trouvee = trouver_equipe(input_nom, equipes_disponibles)
joueurs = joueurs_par_equipe(team, equipe_trouvee)
print(f"Les joueurs de l'équipe {correspondance_equipes.get(equipe_trouvee)} sont les suivants :")
print(joueurs['NAME'])

def cinq_majeur(dataframe, joueurs):
    # Trier les joueurs par ordre décroissant de temps de jeu moyen (MPG)
    equipe_triee = joueurs.sort_values(by='MPG', ascending=False)
    # Sélectionner les cinq premiers joueurs avec le temps de jeu moyen le plus élevé
    cinq_majeur = equipe_triee.head(5)
    return cinq_majeur
cinq_joueurs = cinq_majeur(donnees, joueurs)
print("Les cinq joueurs ayant le temps de jeu moyen le plus élevé dans l'équipe sont :")
print(cinq_joueurs[['NAME', 'MPG']])

Les joueurs de l'équipe None sont les suivants :
Series([], Name: NAME, dtype: object)
Les cinq joueurs ayant le temps de jeu moyen le plus élevé dans l'équipe sont :
Empty DataFrame
Columns: [NAME, MPG]
Index: []


Voici ci-dessous un calcul du gamescore basé sur la formule de John Hollinger:

| Statistique attendue          | Variable correspondante dans les données   | Description                                                                         |
|-------------------------------|--------------------------------------------|-------------------------------------------------------------------------------------|
| pts (float)                   | PPG (Points Per Game)                      | Points marqués par le joueur.                                                      |
| fg (float)                    | FG (Field Goals Made)                      | Paniers réussis par le joueur.                                                     |
| fga (float)                   | FGA (Field Goals Attempted)                | Tentatives de panier du joueur.                                                    |
| fta (float)                   | FTA (Free Throws Attempted)                | Lancers francs tentés par le joueur.                                               |
| ft (float)                    | FT (Free Throws Made)                      | Lancers francs réussis par le joueur.                                               |
| orb (float)                   | ORB (Offensive Rebounds)                   | Rebonds offensifs du joueur.                                                       |
| drb (float)                   | DRB (Defensive Rebounds)                   | Rebonds défensifs du joueur.                                                       |
| stl (float)                   | SPG (Steals Per Game)                      | Interceptions réalisées par le joueur.                                              |
| ast (float)                   | APG (Assists Per Game)                     | Passes décisives du joueur.                                                         |
| blk (float)                   | BPG (Blocks Per Game)                      | Contres réalisés par le joueur.                                                     |
| pf (float)                    | PF (Personal Fouls)                        | Fautes personnelles commises par le joueur.                                         |
| tov (float)                   | TPG (Turnovers Per Game)                   | Balles perdues par le joueur.                                                       |

Par ailleurs, on supposeras:

$$ \text{FGA} = \text{2PA} + \text{3PA}$$

Cependant, il est important de noter que dans certains cas, une tentative de tir à trois points peut être suivie d'une faute, ce qui conduirait à une tentative de lancer franc supplémentaire (FTA). Dans de tels cas, la tentative de tir à trois points ne serait pas comptabilisée dans les FGA, mais elle serait incluse dans les FTA.

In [48]:
def calculate_game_score(pts, fg, fga, ftm, fta, orb, drb, stl, ast, blk, pf, tov):
    game_score = (
        pts + 0.4 * fg + 0.7 * orb + 0.3 * drb + stl + 0.7 * ast + 0.7 * blk
        - 0.7 * fga - 0.4 * (fta-ftm) - 0.4 * pf - tov
    )
    return game_score
def game_score_joueur(dataframe, dataframe2, nom_joueur):
    # Récupérer les données du joueur spécifié
    joueur = dataframe[dataframe['NAME'] == nom_joueur]
    joueur_bis = dataframe2[dataframe2['Player'] == nom_joueur]
    if joueur.empty or joueur_bis.empty:
        print("Le joueur spécifié n'a pas été trouvé.")
        return None
    
    
    # Convertir les valeurs pertinentes en float ou int
    pts = float(joueur_bis['PTS'].iloc[0])
    fga = float(joueur_bis['FGA'].iloc[0])
    fg_percent = float(joueur_bis['FG%'].iloc[0])/100
    ftm = float(joueur_bis['FTM'].iloc[0])
    fta = float(joueur_bis['FTA'].iloc[0])
    ft_percent = float(joueur_bis['FT%'].iloc[0])/100
    orb = float(joueur_bis['OREB'].iloc[0])
    drb = float(joueur_bis['DREB'].iloc[0])
    stl = float(joueur_bis['STL'].iloc[0])
    ast = float(joueur_bis['AST'].iloc[0])
    blk = float(joueur_bis['BLK'].iloc[0])
    pf = float(joueur_bis['PF'].iloc[0])
    tov = float(joueur_bis['TOV'].iloc[0])
    fg = fg_percent*fga
    # Afficher les statistiques du joueur
    # TODO : Supprimer plus tard cette partie
    #print(f"Stats pour {nom_joueur}:")
    #print(f"Points : {pts}")
    #print(f"Pourcentage de réussite aux tirs : {fg_percent}")
    #print(f"Paniers réussis : {fg}")
    #print(f"Paniers tentés : {fga}")
    #print(f"Pourcentage de réussite aux lancers francs : {ft_percent}")
    #print(f"Lancers francs réussis : {ftm}")
    #print(f"Lancers francs tentés : {fta}")
    #print(f"Rebonds offensifs : {orb}")
    #print(f"Rebonds défensifs : {drb}")
    #print(f"Interceptions : {stl}")
    #print(f"Passes décisives : {ast}")
    #print(f"Contres : {blk}")
    #print(f"Fautes personnelles : {pf}")
    #print(f"Balles perdues : {tov}")

    # Calcul du Game Score
    game_score = calculate_game_score(pts, fg, fga, ftm, fta, orb, drb, stl, ast, blk, pf, tov)
    return game_score

# Chemin vers le fichier CSV
chemin = "NBA Stats official 2022-3.csv"
# Importer les données depuis le fichier CSV
donnees_bis = pd.read_csv(chemin)

# Calcul du Game Score pour le joueur spécifié
game_score = game_score_joueur(donnees, donnees_bis, joueur_trouve)
print(f"Le Game Score de {joueur_trouve} est de : {game_score}")

Le Game Score de LeBron James est de : 1234.8999999999996


On remarque que les résultats obtenus sont n'importe quoi ! mais je ne sais pas où est l'erreur, les données devraient être bonnes.... On peut ceci dit comparer les joueurs avec ces données puisqu'elles sont simplement sur toute la saison.

On dispose maintenant du cinq majeur de deux équipes choisies que l'ont peut comparer en s'appuyant sur le gamescore pour trouver un pourcentage de victoire.

In [49]:
import plotly.graph_objs as go

def cinq_majeur(dataframe, equipe):
    equipe_triee = equipe.sort_values(by='MPG', ascending=False)
    cinq_majeur = equipe_triee.head(5)
    return cinq_majeur

# Obtenez le cinq majeur des deux équipes
cinq_majeur_equipe1 = cinq_majeur(donnees, equipe_joueur)
cinq_majeur_equipe2 = cinq_majeur(donnees, joueurs)

# Récupérer les noms des joueurs pour chaque équipe
noms_joueurs_equipe1 = cinq_majeur_equipe1['NAME'].tolist()
noms_joueurs_equipe2 = cinq_majeur_equipe2['NAME'].tolist()

# Calculer le Game Score pour chaque joueur de chaque équipe
game_scores_equipe1 = [game_score_joueur(donnees, donnees_bis, joueur) for joueur in noms_joueurs_equipe1]
game_scores_equipe2 = [game_score_joueur(donnees, donnees_bis, joueur) for joueur in noms_joueurs_equipe2]

# Créer les traces pour chaque équipe
trace_equipe1 = go.Bar(
    x=noms_joueurs_equipe1,
    y=game_scores_equipe1,
    name='Equipe 1'
)

trace_equipe2 = go.Bar(
    x=noms_joueurs_equipe2,
    y=game_scores_equipe2,
    name='Equipe 2'
)

# Créer la figure
fig = go.Figure(data=[trace_equipe1, trace_equipe2])

# Mise en forme du layout
fig.update_layout(
    title='Comparaison des Game Scores des cinq majeurs des deux équipes',
    xaxis=dict(title='Joueurs'),
    yaxis=dict(title='Game Score'),
    barmode='group'
)

# Afficher la figure
fig.show()


On a ainsi un premier graphique qui peut nous indiquer des rapports de force entre différentes équipes. Attention, le diagramme affiché ne fait apparaître que le cinq majeur de chaque équipe et les écarts de force peuvent être compensés par de bons joueurs sortant du banc. Il conviendra de les considérer d'autre part et de comparer les pourcentages de victoire obtenus dans les deux cas.

In [50]:
score_equipe1 = sum(game_scores_equipe1)
print("La somme des Game Scores des joueurs de l'équipe 1 est :", score_equipe1)

score_equipe2 = sum(game_scores_equipe2)
print("La somme des Game Scores des joueurs de l'équipe 2 est :", score_equipe2)

# Calcul très simplifié d'un pourcentage de victoire possible : 
victoire = 100 * score_equipe1 / (score_equipe1 + score_equipe2) 
# Conserver seulement les quatre premiers chiffres 
vic = round(victoire, 2)
print("L'équipe 1 a un pourcentage de victoire face à l'équipe", equipe_trouvee, "de ", vic, "%.")


La somme des Game Scores des joueurs de l'équipe 1 est : 4559.8684
La somme des Game Scores des joueurs de l'équipe 2 est : 0
L'équipe 1 a un pourcentage de victoire face à l'équipe None de  100.0 %.


A la fin de la saison régulière, il y a en NBA les Playoffs qui sont des affrontements en BO7 (Best Of 7). Avec le pourcentage de victoire calculé on peut estimer les probabilités de chaque issue du BO7.

In [51]:
import math

# Probabilité de victoire de l'équipe 1 (en pourcentage)
proba = vic / 100

# Fonction pour calculer le coefficient binomial (combinaison)
def binomial_coefficient(n, k):
    return math.comb(n, k)

# Résultats possibles du BO7
resultats1 = [(4, 3-i) for i in range(4)]
resultats2 = [(i, 4) for i in range(4)]

# Calcul des probabilités de chaque résultat possible pour l'équipe 1
proba1 = {}
for equipe1_wins, equipe2_wins in resultats1:
    prob = binomial_coefficient(equipe1_wins+equipe2_wins, equipe2_wins) * ((1-proba) ** equipe2_wins) * ((proba) ** (equipe1_wins))
    proba1[(equipe1_wins, equipe2_wins)] = prob

# Calcul des probabilités de chaque résultat possible pour l'équipe 2
proba2 = {}
for equipe1_wins, equipe2_wins in resultats2:
    prob = binomial_coefficient(equipe1_wins+equipe2_wins, equipe1_wins) * ((proba) ** equipe1_wins) * ((1-proba) ** (equipe2_wins))
    proba2[(equipe1_wins, equipe2_wins)] = prob

# Affichage des probabilités de chaque résultat possible
for resultat, prob in proba2.items():
    print(f"Probabilité que l'équipe 1 perde {resultat[0]}-{resultat[1]} : {prob:.4f}")
for resultat, prob in proba1.items():
    print(f"Probabilité que l'équipe 1 gagne {resultat[0]}-{resultat[1]} : {prob:.4f}")



Probabilité que l'équipe 1 perde 0-4 : 0.0000
Probabilité que l'équipe 1 perde 1-4 : 0.0000
Probabilité que l'équipe 1 perde 2-4 : 0.0000
Probabilité que l'équipe 1 perde 3-4 : 0.0000
Probabilité que l'équipe 1 gagne 4-3 : 0.0000
Probabilité que l'équipe 1 gagne 4-2 : 0.0000
Probabilité que l'équipe 1 gagne 4-1 : 0.0000
Probabilité que l'équipe 1 gagne 4-0 : 1.0000


On remarque que la somme des probabilités calculées est supérieure à 1. Cela est lié à la manière dont on a calculé ces probabilités, en séparant le calcul en deux selon si l'équipe 1 gagne ou perd. Cependant, les rapports entre les probabilités sont bons et on peut donc normaliser les probabilités obtenues en les divisant par la somme des probabilités.

In [52]:
#On calcule la somme des probabilités
somme =0
for resultat, prob in proba1.items():
    somme +=prob
for resultat, prob in proba2.items():
    somme +=prob
print(somme)

#On normalise les probabilités obtenues et on les ordonne dans une liste
proba = []
for resultat, prob in proba2.items():
    proba.append(prob/somme)
for resultat, prob in proba1.items():
    proba.append(prob/somme)

print(proba)

1.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]


In [53]:
import plotly.graph_objects as go

# Définition des résultats possibles
resultats = ["0-4", "1-4", "2-4", "3-4", "4-3", "4-2", "4-1", "4-0"]

# Création du graphique en barres
fig = go.Figure(go.Bar(x=resultats, y=proba))

# Personnalisation du graphique
fig.update_layout(
    title="Probabilités des issues possibles dans un BO7",
    xaxis_title="Résultat",
    yaxis_title="Probabilité",
    yaxis_tickformat=".2%",
)

# Affichage du graphique
fig.show()
